In [0]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project clever-aleph-203411

Updated property [core/project].


In [0]:
# Download the file from a given Google Cloud Storage bucket.
!gsutil cp gs://colab-bucket-f1618a8a-3e8c-11e9-ac44-0242ac1c0002/dataset_224x224.h5 dataset.h5

Copying gs://colab-bucket-f1618a8a-3e8c-11e9-ac44-0242ac1c0002/dataset_224x224.h5...
/ [1 files][  2.1 GiB/  2.1 GiB]   21.4 MiB/s                                   
Operation completed over 1 objects/2.1 GiB.                                      


In [0]:
!mkdir models

In [0]:
import keras
from keras.models import Sequential, model_from_json, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle
import h5py

Using TensorFlow backend.


In [0]:
with h5py.File('dataset.h5', 'r') as f:
  x = f['x'][:]
  y = f['y'][:]
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
  del x
  del y

In [0]:
y.shape

(9726, 2)

In [0]:
res_model = ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(224,224,1), pooling=None)


Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [0]:
x=res_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
preds=Dense(2,activation='linear')(x) #final layer with linear activation

model=Model(inputs=res_model.input,outputs=preds)

In [0]:
# initiate optimizer
opt = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#opt = keras.optimizers.RMSprop(lr=0.001)

model.compile(loss='mse', optimizer=opt, metrics=['mae'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=12, restore_best_weights=True)
#sm = SaveModel()
rp = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0)

In [0]:
batch_size = 64
epochs = 100

In [0]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, shuffle=True, verbose=1, callbacks=[es, rp])

Train on 6224 samples, validate on 1556 samples
Epoch 1/100
6224/6224 [==============================] - 168s 27ms/step - loss: 169.1300 - mean_absolute_error: 10.1529 - val_loss: 246.2219 - val_mean_absolute_error: 12.2936
Epoch 2/100
6224/6224 [==============================] - 165s 26ms/step - loss: 132.8889 - mean_absolute_error: 9.0126 - val_loss: 203.9331 - val_mean_absolute_error: 11.4084
Epoch 3/100
6224/6224 [==============================] - 165s 26ms/step - loss: 118.8605 - mean_absolute_error: 8.5919 - val_loss: 129.4071 - val_mean_absolute_error: 8.9939
Epoch 4/100
6224/6224 [==============================] - 164s 26ms/step - loss: 117.0571 - mean_absolute_error: 8.5162 - val_loss: 357.5388 - val_mean_absolute_error: 13.0000
Epoch 5/100
6224/6224 [==============================] - 165s 26ms/step - loss: 125.3855 - mean_absolute_error: 8.6947 - val_loss: 129.6206 - val_mean_absolute_error: 8.7283
Epoch 6/100
6224/6224 [==============================] - 165s 26ms/step - loss

In [0]:
model.save('resnet_rnd_100.h5')